In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats
                                           
                                                  
from general_tools.in_out.basics import create_dir, files_in_subdirs, pickle_data, unpickle_data
from general_tools.simpletons import select_first_last_and_k
from general_tools.simpletons import indices_in_iterable

from geo_tool import Point_Cloud


from tf_lab.autopredictors.scripts.virt_scan_data import load_partial_pointclouds
import tf_lab.autopredictors.scripts.virt_scan_data as vscan

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
loss = 'emd'
incomplete_n_samples = 2048
complete_n_samples = 4096
max_training_epochs = 100
do_training = True
load_model = False
report_cov_and_acc = False
do_detailed_plotting = False

experiment_tag = raw_input('Give an experiment tag-ID: ')

experiment_name = 'all_classes_in_dai' + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss + '_' + experiment_tag


top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

complete_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/raw', \
                             str(complete_n_samples))

partial_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Partial_PCs/Angela_synthetic', str(incomplete_n_samples))

n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3]

train_dir = osp.join(top_data_dir, 'OUT/iccv/nn_models/')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

Give an experiment tag-ID: original


'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/all_classes_in_dai_2048_4096pts_emd_original'

In [7]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, cmpl_syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=50)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )
cmpl_unames = np.array(cmpl_model_names + '.' + cmpl_syn_ids, dtype=object)

In [8]:
train_list = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Partial_PCs/Angela_synthetic/train_test_splits/train_models_with_scan_id.txt'
partial_pclouds, partial_model_ids, partial_syn_ids = load_partial_pointclouds(train_list, partial_pclouds_path, 50)

partial_model_names = np.array([i[0] for i in partial_model_ids], dtype=object)
partial_scan_ids = np.array([i[1] for i in partial_model_ids], dtype=object)
partial_unames = np.array(partial_model_names + '.' + partial_scan_ids + '.' + partial_syn_ids, dtype=object)
partial_matching_names = np.array(partial_model_names + '.' + partial_syn_ids, dtype=object)

matching_index = indices_in_iterable(cmpl_unames, partial_matching_names)
assert(np.alltrue(matching_index != -1))
full_pclouds_matched = cmpl_pclouds[matching_index]

train_data = PointCloudDataSet(full_pclouds_matched, noise=partial_pclouds, labels=partial_unames)
train_data, val_data = vscan.make_validation_from_train_data(train_data, 0.05)

In [32]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:    
    encoder_args = {'n_filters': [64, 128, 1024],
                    'filter_sizes': [1, 1, 1],
                    'strides': [1, 1, 1],
                    'dropout_prob': 0.5
                   }
    
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)]}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 1,
                learning_rate = 0.0005,
                z_rotate = False,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)

In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '27.7351', 'loss=', '290.127079349')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/all_classes_in_dai_2048_4096pts_emd_original/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '26.9143', 'loss=', '252.520612349')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/all_classes_in_dai_2048_4096pts_emd_original/models.ckpt-2 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0003', 'training time (minutes)=', '27.2460', 'loss=', '241.255106144')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_models/all_classes_in_dai_2048_4096pts_emd_original/models.ckpt-3 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0004', 'training time (minutes)=', '27.2280', 'loss=', '235.114763910')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iccv/nn_

In [ ]:
# Use validation to pick best_epoch, i.e., the one with the smallest error.
save_validation_file = osp.join(conf.train_dir, 'validation_stats.pickle')

In [ ]:
saved_epochs = np.array(read_saved_epochs(conf.train_dir))
val_stats = eval_model(ae, conf, val_data, epochs=saved_epochs)
val_loss = np.min(val_stats[:,1])
best_epoch = int(val_stats[np.argmin(val_stats[:,1]), 0])
print best_epoch, val_loss

pickle_data(save_validation_file, best_epoch, val_loss)

# Report loss on test data at optimal (per-validation) epoch
best_epoch, val_loss = unpickle_data(save_validation_file)
print eval_model(ae, conf, test_data, epochs=[best_epoch])[0][1]